# Analiza obwodu elektrycznego - nadokreślony układ równań

importujemy potrzebne biblioteki

In [311]:
import numpy as np
import networkx as nx
from pyvis.network import Network
from collections import deque


wczytujemy graf z pliku

In [312]:
def load_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        G = nx.Graph()
        for line in lines:
            u, v, resistance = line.split()
            G.add_edge(int(u), int(v), res=float(resistance))
    return G

Znajdujemy natężenia prądów w obwodzie metodą potencjałów węzłowych
W tym celu musimy dla każdego węzła obliczyć jego potencjał elektryczny na podstawie prawa Kirchhoffa dla węzłów oraz prawa Ohma dla gałęzi.
Następnie dla każdej gałęzi obliczamy natężenie prądu na podstawie różnicy potencjałów oraz rezystancji.

n - liczba węzłów
m - liczba gałęzi
Macierz konduktancji A:
(dla każdego sąsiada węzła i)
- A[i][i] = 1 / rezystancja, czyli  sumujemy przewodności wszystkich gałęzi wychodzących z węzła.
- A[i][j] = -1 / rezystancja, czyli odejmujemy wpływ innych węzłów

Wektor wymuszeń b:
- b[i] = E, dla węzła źródłowego
- b[i] = 0, dla pozostałych węzłów

Rozwiązujemy układ równań A * V = b, gdzie V to wektor potencjałów węzłowych
Następnie obliczamy różnicę potencjałów dla każdej gałęzi i na jej podstawie natężenie prądu ze wzoru I = V_diff / rezystancja

In [313]:

def solve_circuit(file, s, t, E):
    G = load_from_file(file)
    nodes = list(G.nodes)
    
    masa = t  # Ustalony węzeł odniesienia
    n = len(nodes)

    # Macierz konduktancji i wektor wymuszeń
    A = np.zeros((n, n))
    b = np.zeros(n)

    for i, node in enumerate(nodes):
        if node == masa:
            A[i][i] = 1
            b[i] = 0
        else:
            for neighbor in G.neighbors(node):
                j = nodes.index(neighbor)
                res = G.get_edge_data(node, neighbor)['res']
                A[i][i] += 1 / res
                A[i][j] -= 1 / res
    # for i in normal_nodes:
    #     nd_idx = nodes_idx[i]
    #     for j in G.neighbors(i):
    #         if (i, j) == (s, t) or (j, i) == (s, t):
    #             continue
    #         R = G[i][j]['R']
    #         conductance = 1.0 / R
    #         if j in nodes_idx:
    #             A[nd_idx, nodes_idx[j]] -= conductance
    #         else:
    #             b[nd_idx] += conductance * special[j]
    #         A[nd_idx, nd_idx] += conductance
    #         
    # Wymuszenie źródła napięciowego między s a t
    b[nodes.index(s)] = E 

    # Rozwiązanie układu równań
    V = np.linalg.solve(A, b)

    # Przypisanie potencjałów do węzłów
    for i, node in enumerate(nodes):
        G.nodes[node]['V'] = V[i]

    # Obliczenie prądów w gałęziach
    for u, v, data in G.edges(data=True):
        V_diff = V[nodes.index(u)] - V[nodes.index(v)]
        I = V_diff / data['res']
        G[u][v]['V'] = V_diff
        G[u][v]['I'] = I

    return G

In [314]:
def convert_to_directed(G, s):
    Gdir = nx.DiGraph()
    for node in G.nodes:
        Gdir.add_node(node, V=G.nodes[node]['V'])
    
    visited = set()
    queue = deque([s])
    
    while queue:
        u = queue.popleft()
        visited.add(u)
        
        for neighbor in G.neighbors(u):
            if neighbor not in visited:
                data = G.get_edge_data(u, neighbor)
                if data['I'] > 0:
                    Gdir.add_edge(u, neighbor, **data)
                else:
                    data['I'] = -data['I']
                    Gdir.add_edge(neighbor, u, **data)
                queue.append(neighbor)
    
    return Gdir

def set_direction(G):
    """jeśli natężenie prądu jest ujemne, to zmieniamy kierunek gałęzi"""
    edges = list(G.edges(data=True))
    for fr, to, data in edges:
        if data['I'] < 0:
            data['I'] = -data['I']
            G.remove_edge(fr, to)
            G.add_edge(to, fr, **data)

In [315]:
low_color = [0, 255, 0]
high_color = [255, 0, 0]
def draw_circuit(G):
    # print(G.edges(data=True))
    net = Network(height='750px', width='100%', notebook=True, cdn_resources="remote")
    for node in G.nodes:
        net.add_node(node, label=str(node) + f" V: {G.nodes[node]['V']:.2f}")
        # net.add_node(node, label=str(node))
    for edge in G.edges(data=True):
        u, v, data = edge
        color = f"rgb({int(low_color[0] + (high_color[0] - low_color[0]) * (abs(data['I'] )/ 10))}, {int(low_color[1] + (high_color[1] - low_color[1]) * (abs(data['I'] )/ 10))}, 0)"
        if 'I'  in data:
            direction = 'to' if data['I'] > 0 else 'from'
            # net.add_edge(u, v, label=f"I: {data['I']:.2f}A, R: {data['res']:.2f} V: {data['V']:.2f}", color=color, width=5, arrows='from')
            net.add_edge(u, v, label=f"I: {data['I']:.2f}A, R: {data['res']:.2f} ", color=color, width=5, arrows=direction)
        else:
            print("No I")
            net.add_edge(u, v, label=f"R: {data['res']:.2f}")
    net.toggle_physics(True)
    net.show('circuit.html')


In [316]:
EPS = 1e-10

def check_first_kirchoff_law(G,s,t):
    verts = [0]*len(G.nodes)
    for e in G.edges:
        i = G[e[0]][e[1]]['I']
        verts[e[0]] -= i
        verts[e[1]] += i
        
    for i, curr in enumerate(verts):
        print(i, curr)
        if i == s or i == t:
            continue
        if abs(curr) > EPS:
            return False
    
    return True

def check_second_kirchoff_law(G, Gundir, s, t, E):
    cycles = nx.simple_cycles(Gundir)
    edges = G.edges

    for cycle in cycles:
        cycle_edges=zip(cycle,cycle[1:]+[cycle[0]])
        cycle_sum = 0
        print(cycle)
        if s in cycle or t in cycle:
            continue
        for e in cycle_edges:
            if e == (s,t):
                cycle_sum -= E
            elif e == (t,s):
                cycle_sum += E
            else:
                if e in edges:
                    cycle_sum += G[e[0]][e[1]]['I'] * G[e[0]][e[1]]['res']
                else:
                    cycle_sum -= G[e[1]][e[0]]['I'] * G[e[1]][e[0]]['res']
        print(cycle_sum)
        if abs(cycle_sum) > EPS:
            return False
    
    return True


In [317]:
def TEST(Gdir, Gundir, s, t, E):
    print("I Kirchoff law:", "PASSED" if check_first_kirchoff_law(Gdir,s,t) else "FAILED!!")
    # print("II Kirchoff law:", "PASSED" if check_second_kirchoff_law(Gdir, Gundir, s, t, E) else "FAILED!!")

In [318]:

result= solve_circuit("graf.txt", 21, 9,10)
# gdir = convert_to_directed(result, 21)
draw_circuit(result)
graph = load_from_file("graf.txt")
TEST(result, graph,  21, 9,10)
# draw_circuit(solve_circuit(sample_graph, 1, 0, 10))


    

circuit.html
0 9.992007221626409e-16
1 -8.881784197001252e-16
2 -3.608224830031759e-16
3 -3.3306690738754696e-16
4 -1.5543122344752192e-15
5 4.163336342344337e-16
6 1.0269562977782698e-15
7 -2.1094237467877974e-15
8 -9.992007221626409e-16
9 10.000000000000007
10 -1.5543122344752192e-15
11 -6.661338147750939e-16
12 -2.3592239273284576e-15
13 1.4432899320127035e-15
14 3.552713678800501e-15
15 1.7763568394002505e-15
16 -2.858824288409778e-15
17 1.1102230246251565e-15
18 4.551914400963142e-15
19 3.552713678800501e-15
20 -3.605449272470196e-14
21 -9.999999999999986
22 5.329070518200751e-15
23 4.9682480351975755e-15
24 -2.7755575615628914e-15
I Kirchoff law: PASSED
